# Run REopt API

1. [Single API call using pre-defined inputs file (Melbourne, FL)](#singlepredefined)
2. [Extract one API result](#extract-one)
3. [Single API call using automated input file](#custom_input)
4. [Loop through the entire spreadsheet](#run_all)

## Initialization

In [318]:
import pandas as pd
import json
import requests
from pathlib import Path
from datetime import datetime
from fuzzywuzzy import fuzz  # conda install fuzzywuzzy
from src.post_and_poll import get_api_results, get_run_uuid
from src.results_poller import poller
from API_KEYS import API_KEY, GOOGLE_API_KEY  # Get this private file from Gerry
import urllib3
urllib3.disable_warnings() # InsecureRequestWarning - need to add certificate

In [81]:
inputs = Path('inputs')
outputs = Path('outputs')

## First, run an API call using a pre-defined inputs file <a class="anchor" id="singlepredefined"></a>
The `VA_inputs2.json` file was created by manually creating in a run using the online tool, then selecting [Download JSON](https://reopt.nrel.gov/tool/results/run-inputs.json?id=657b76b6-5cc8-490a-8a43-b6c91660f08f) from the "Saved Evaluations".

In [82]:
# Load a json into a python dictionary
with open(inputs / 'VA_inputs2.json', 'r') as fp:
    post = json.load(fp)

### Query the API

A query consists of 2 parts:
1. POST - basically ask the server to start computing a result
2. poll - keep asking the server "are you done?" every 5 seconds until the server responds with the result.

This takes ~30s.

Note, the `api_url` in the `get_api_results` function below calls the **production server** hosted API (master/main branch/version, publicly accessible)

Also, a `main.log` file is created in the current directory on your computer, in case you want to reference the job id's and stuff.

In [11]:
outputs_file_name = "results_file.json"
root_url = "https://developer.nrel.gov/api/reopt/stable"
api_response = get_api_results(post=post, 
                               API_KEY=API_KEY, 
                               api_url=root_url, 
                               results_file=outputs / outputs_file_name, 
                               run_id=None)

main         INFO     Response OK from https://developer.nrel.gov/api/reopt/stable/job/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L.
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/25c1fc1a-47a8-4fce-915c-fa49012443b1/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/results_file.json


### If you get disconnected
If you think POST ran but got disconnected during polling, copy the run_uuid from the log above to manually GET the results:

In [76]:
if False:
    run_uuid = api_response["outputs"]["Scenario"]["run_uuid"]
    results_url = root_url + '/job/' + run_uuid + '/results/?api_key=' + API_KEY
    resp = requests.get(url=results_url, verify=False)
    api_response = json.loads(resp.text)

## Test extracting one result <a class="anchor" id="extract-one"></a>
Extracts the API result into the GP spreadsheet

In [83]:
# First load the spreadsheet and visualize it as a sanity check
pd.set_option('display.max_columns', 100)
template = pd.read_excel(outputs / 'REopt_Master_Spreadsheet_template.xlsx', header=[0, 1, 2])
template.iloc[33:35]

Unnamed: 0_level_0 Unnamed: 1_level_0             Inputs  \
         Point Person           Reviewer               cbsa   
   Unnamed: 0_level_2 Unnamed: 1_level_2 Unnamed: 2_level_2   
33           Virginia              Naomi            26900.0   
34           Virginia              Naomi            37340.0   

                                          \
            City/MSA               State   
   Unnamed: 3_level_2 Unnamed: 4_level_2   
33       Indianapolis                 IN   
34          Melbourne                 FL   

                                                                          \
                                           Utility  System Capacity (kW)   
                                 Unnamed: 5_level_2   Unnamed: 6_level_2   
33  AES Indiana (fka Indianapolis Power & Light Co)             1,000 kW   
34                  Florida power and light company              2000 kW   

                                                                   \
   Adjusted System Capacity (kW) State system capacity limit (kW)   
              Unnamed: 7_level_2               Unnamed: 8_level_2   
33                        1000.0                              0.0   
34                        2000.0                             10.0   

                                       \
   Min btw state and utility NEM (kW)   
                   Unnamed: 9_level_2   
33                                  0   
34                                 10   

                                              \
   System capacity *100 (for 100 apartments)   
                         Unnamed: 10_level_2   
33                                       0.0   
34                                    1000.0   

               Energy Production & Fuel Use  \
   Recommended Solar Installation size (kW)   
                                        N/A   
33                                      NaN   
34                                    213.0   

                                          \
   Avg Annual PV Energy Produciton (kWh)   
                               FINANCIAL   
33                                   NaN   
34                              335063.0   

                                              \
   Annual Renewable Electricity (% of total)   
                                   FINANCIAL   
33                                       NaN   
34                                       1.0   

                       Climate Emissions  \
   % Reduction in CO2 Emissions from BAU   
                               FINANCIAL   
33                                   NaN   
34                                0.9997   

                                                                   \
   Lifecycle Costs of Climate Emissions ($) (difference from BAU)   
                                                       DIFFERENCE   
33                                                NaN               
34                                          -759766.0               

                                                Health Emissions  \
   Lifecycle Costs of Health Emissions ($) (difference from BAU)   
                                                      DIFFERENCE   
33                                                NaN              
34                                           -95554.0              

   Life Cycle Cost Breakdown (25 year period)  Summary Financial Metrics  \
           Total Utility Electricity Cost ($) Total Life Cycle Costs ($)   
                                   DIFFERENCE                  FINANCIAL   
33                                        NaN                        NaN   
34                                  -628365.0                   415325.0   

                           Climate Emissions       Health Emissions  \
         IRR Lifecycle Total          t CO2  Lifecycle Total t Nox    
   FINANCIAL                      DIFFERENCE             DIFFERENCE   
33       NaN                             NaN                    NaN   
34     0.155         

In [316]:
# Now extract the data from the json into a dataframe
OUTPUTS = template.columns[11:]  # which columns to edit
print('columns to edit are:', OUTPUTS.get_level_values(1))
def extract_info_into_df(data, rowi, api_response):
    # API documentation here:
    # https://github.com/NREL/REopt_API/blob/master/reo/nested_outputs.py
    site = api_response['outputs']['Scenario']['Site']
    pv = site['PV']
    financial = site['Financial']
    for o in OUTPUTS:
        data.loc[rowi, o] = None
    data.loc[rowi, OUTPUTS[0]] = pv['size_kw']
    data.loc[rowi, OUTPUTS[1]] = pv['average_yearly_energy_produced_kwh']
    data.loc[rowi, OUTPUTS[2]] = site['annual_total_renewable_energy_pct'] # total of electricity or energy?
    data.loc[rowi, OUTPUTS[3]] = site['lifecycle_emissions_reduction_CO2_pct']
    data.loc[rowi, OUTPUTS[4]] = site['lifecycle_emissions_cost_CO2'] - site['lifecycle_emissions_cost_CO2_bau']
    data.loc[rowi, OUTPUTS[5]] = site['lifecycle_emissions_cost_Health'] - site['lifecycle_emissions_cost_Health_bau']
    utility_cost = financial['lcc_us_dollars'] - financial['net_capital_costs_plus_om_us_dollars']
    data.loc[rowi, OUTPUTS[6]] = utility_cost - financial['lcc_bau_us_dollars']
    data.loc[rowi, OUTPUTS[7]] = financial['lcc_us_dollars']
    data.loc[rowi, OUTPUTS[8]] = financial['irr_pct']
    data.loc[rowi, OUTPUTS[9]] = site['lifecycle_emissions_tCO2'] - site['lifecycle_emissions_tCO2_bau']
    data.loc[rowi, OUTPUTS[10]] = site['lifecycle_emissions_tNOx'] - site['lifecycle_emissions_tNOx_bau']
    data.loc[rowi, OUTPUTS[11]] = site['lifecycle_emissions_tSO2'] - site['lifecycle_emissions_tSO2_bau']
    data.loc[rowi, OUTPUTS[12]] = site['lifecycle_emissions_tPM25'] - site['lifecycle_emissions_tPM25_bau']
    data.loc[rowi, OUTPUTS[13]] = "Virginia" # Completed By
    data.loc[rowi, OUTPUTS[14]] = datetime.now().strftime("%m/%d/%Y") # Date Completed


new_data = template.copy()                        # don't over-write the old data
extract_info_into_df(new_data, 34, api_response)  # save the API results into the dataframe
display(new_data.iloc[34:35])                     # Compare the new data...
display(template.iloc[34:35])                     # to the old data


columns to edit are: Index(['Recommended Solar Installation size (kW)',
       'Avg Annual PV Energy Produciton (kWh)',
       'Annual Renewable Electricity (% of total)',
       '% Reduction in CO2 Emissions from BAU',
       'Lifecycle Costs of Climate Emissions ($) (difference from BAU)',
       'Lifecycle Costs of Health Emissions ($) (difference from BAU)',
       'Total Utility Electricity Cost ($)', 'Total Life Cycle Costs ($)',
       'IRR', 'Lifecycle Total          t CO2 ', 'Lifecycle Total t Nox ',
       'Lifecycle Total t SO2 ', 'Lifecycle Total t PM2.5 ',
       'Lifecycle Total t PM2.5 ', 'Lifecycle Total t PM2.5 ',
       'Lifecycle Total t PM2.5 '],
      dtype='object')


Unnamed: 0_level_0 Unnamed: 1_level_0             Inputs  \
         Point Person           Reviewer               cbsa   
   Unnamed: 0_level_2 Unnamed: 1_level_2 Unnamed: 2_level_2   
34           Virginia              Naomi            37340.0   

                                                                           \
            City/MSA               State                         Utility    
   Unnamed: 3_level_2 Unnamed: 4_level_2               Unnamed: 5_level_2   
34          Melbourne                 FL  Florida power and light company   

                                                       \
   System Capacity (kW) Adjusted System Capacity (kW)   
     Unnamed: 6_level_2            Unnamed: 7_level_2   
34              2000 kW                        2000.0   

                                                                        \
   State system capacity limit (kW) Min btw state and utility NEM (kW)   
                 Unnamed: 8_level_2                 Unnamed: 9_level_2   
34                             10.0                                 10   

                                              \
   System capacity *100 (for 100 apartments)   
                         Unnamed: 10_level_2   
34                                    1000.0   

               Energy Production & Fuel Use  \
   Recommended Solar Installation size (kW)   
                                        N/A   
34                                      0.0   

                                          \
   Avg Annual PV Energy Produciton (kWh)   
                               FINANCIAL   
34                                   0.0   

                                              \
   Annual Renewable Electricity (% of total)   
                                   FINANCIAL   
34                                       0.0   

                       Climate Emissions  \
   % Reduction in CO2 Emissions from BAU   
                               FINANCIAL   
34                                   0.0   

                                                                   \
   Lifecycle Costs of Climate Emissions ($) (difference from BAU)   
                                                       DIFFERENCE   
34                                                0.0               

                                                Health Emissions  \
   Lifecycle Costs of Health Emissions ($) (difference from BAU)   
                                                      DIFFERENCE   
34                                                0.0              

   Life Cycle Cost Breakdown (25 year period)  Summary Financial Metrics  \
           Total Utility Electricity Cost ($) Total Life Cycle Costs ($)   
                                   DIFFERENCE                  FINANCIAL   
34                                        0.0                   295697.0   

                           Climate Emissions       Health Emissions  \
         IRR Lifecycle Total          t CO2  Lifecycle Total t Nox    
   FINANCIAL                      DIFFERENCE             DIFFERENCE   
34       NaN                             0.0                    0.0   

                                                                  \
   Lifecycle Total t SO2  Lifecycle Total t PM2.5                  
               DIFFERENCE               DIFFERENCE Completed By    
34                    0.0                      0.0      Virginia   

                                    
                                    
   Date Completed Date Completed.1  
34     2022-11-24             None

Unnamed: 0_level_0 Unnamed: 1_level_0             Inputs  \
         Point Person           Reviewer               cbsa   
   Unnamed: 0_level_2 Unnamed: 1_level_2 Unnamed: 2_level_2   
34           Virginia              Naomi            37340.0   

                                                                           \
            City/MSA               State                         Utility    
   Unnamed: 3_level_2 Unnamed: 4_level_2               Unnamed: 5_level_2   
34          Melbourne                 FL  Florida power and light company   

                                                       \
   System Capacity (kW) Adjusted System Capacity (kW)   
     Unnamed: 6_level_2            Unnamed: 7_level_2   
34              2000 kW                        2000.0   

                                                                        \
   State system capacity limit (kW) Min btw state and utility NEM (kW)   
                 Unnamed: 8_level_2                 Unnamed: 9_level_2   
34                             10.0                                 10   

                                              \
   System capacity *100 (for 100 apartments)   
                         Unnamed: 10_level_2   
34                                    1000.0   

               Energy Production & Fuel Use  \
   Recommended Solar Installation size (kW)   
                                        N/A   
34                                    213.0   

                                          \
   Avg Annual PV Energy Produciton (kWh)   
                               FINANCIAL   
34                              335063.0   

                                              \
   Annual Renewable Electricity (% of total)   
                                   FINANCIAL   
34                                       1.0   

                       Climate Emissions  \
   % Reduction in CO2 Emissions from BAU   
                               FINANCIAL   
34                                0.9997   

                                                                   \
   Lifecycle Costs of Climate Emissions ($) (difference from BAU)   
                                                       DIFFERENCE   
34                                          -759766.0               

                                                Health Emissions  \
   Lifecycle Costs of Health Emissions ($) (difference from BAU)   
                                                      DIFFERENCE   
34                                           -95554.0              

   Life Cycle Cost Breakdown (25 year period)  Summary Financial Metrics  \
           Total Utility Electricity Cost ($) Total Life Cycle Costs ($)   
                                   DIFFERENCE                  FINANCIAL   
34                                  -628365.0                   415325.0   

                           Climate Emissions       Health Emissions  \
         IRR Lifecycle Total          t CO2  Lifecycle Total t Nox    
   FINANCIAL                      DIFFERENCE             DIFFERENCE   
34     0.155                         -4835.0                  -3.11   

                                                                  \
   Lifecycle Total t SO2  Lifecycle Total t PM2.5                  
               DIFFERENCE               DIFFERENCE Completed By    
34                  -2.59                    -0.27      Virginia   

                                    
                                    
   Date Completed Date Completed.1  
34     2022-11-23              NaN

## Run custom/programmatic input <a class="anchor" id="custom_input"></a>

First, fetch NREL URDB input

In [278]:
# Test NREL URDB API
# https://apps.openei.org/services/doc/rest/util_rates/?version=3#request-url

# First define some filters
now, default_expiration = datetime.now(), datetime.timestamp(datetime.now()) + 1
valid_date = lambda x: now < datetime.fromtimestamp(x.get('enddate', default_expiration))
is_time_of_use = lambda x: (('time of use' in x['name'].lower()) or
                            ('time-of-use' in x['name'].lower()) or ('TOU' in x['name']))
default_filters = [valid_date, is_time_of_use]
# default_filters = [valid_date]

# Fetch URDB data
def get_urdb_data(city, filters=default_filters, utility_name_hint=None, labelid=None):
    if labelid is None:
        # First perform a search and filter by valid end-date
        urdb_results = requests.get('https://api.openei.org/utility_rates',
                        params={
                            'version': 'latest',
                            'format': 'json',
                            'api_key': API_KEY,
                            'sector': 'Residential',
                            'address': city,
                        }).json()['items']
        if utility_name_hint is not None:
            a = len(urdb_results)
            print('Utility name filtering: ', utility_name_hint, [r['utility'] for r in urdb_results])
            urdb_results = [r for r in urdb_results if fuzz.partial_ratio(utility_name_hint, r['utility']) > 90]
            print('matches before and after filtering by utility name: ', a, len(urdb_results))
        for filter_ in filters:
            a = len(urdb_results)
            urdb_results = list(filter(filter_, urdb_results))
            print('matches before and after a step of filtering: ', a, len(urdb_results))
        if len(urdb_results) == 0:
            raise RuntimeError('No URDB results found for city: {}'.format(city))
        if len(urdb_results) > 1:  # Need to manually select the utility
            # kwds_to_print = ['utility', 'name', 'description']
            kwds_to_print = ['utility', 'name']
            for i, res in enumerate(urdb_results):
                print(i, res['utility'], res['name'])
                # display({key: res.get(key, 'No {:}'.format(key)) for key in kwds_to_print})
            input_index = int(input('Which utility? (enter number) '))
            urdb_results = [urdb_results[input_index]]
        labelid = urdb_results[0]['label']

    # Now get the data for the selected utility
    urdb_results = requests.get('https://api.openei.org/utility_rates',
                                params={
                                    'version': 'latest',
                                    'format': 'json',
                                    'api_key': API_KEY,
                                    'getpage': labelid,
                                    'detail': 'full',
                                })
    return urdb_results.json()['items'][0]

urdb = get_urdb_data('Melbourne, FL')

matches before and after a step of filtering:  28 2
matches before and after a step of filtering:  2 1


In [279]:
# Fetch lat/long from Google geocode:
def get_lat_long_from_google(city):
    res = requests.get('https://maps.googleapis.com/maps/api/geocode/json',
                       params={
                           'address': city,
                           'key': GOOGLE_API_KEY
                       }).json()
    loc = res['results'][0]['geometry']['location']
    return {'latitude': loc['lat'], 'longitude': loc['lng']}
lat_long = get_lat_long_from_google('Melbourne, FL')
print(lat_long)

{'latitude': 28.0836269, 'longitude': -80.60810889999999}


In [280]:
# API documentation:
# https://github.com/NREL/REopt_API/blob/master/reo/nested_inputs.py
def create_input(city_name, net_metering_limit, urdb=None, lat_long=None, **util_lookup_kwargs):
    if urdb is None:
        urdb = get_urdb_data(city_name, **util_lookup_kwargs)
    if lat_long is None:
        lat_long = get_lat_long_from_google(city_name)

    obj = {"Scenario": {"Site": {}}}
    site = obj["Scenario"]["Site"]
    # query = {"Scenario": {"Site":
    # {}},}
    # Step 1
    # TODO: energy goals
    # Step 2
    site["Storage"] = {"max_kw": "0.0", "max_kwh": "0.0"}
    # Step 3
    site["address"] = city_name.replace(',', '') + ' USA'
    site["latitude"] = lat_long['latitude']
    site["longitude"] = lat_long['longitude']
    site["ElectricTariff"] = {'urdb_response': urdb}
    #   2 - roofspace only
    site["land_acres"] = 0
    site["roof_squarefeet"] = None
    #   3. net metering system capacity
    site["ElectricTariff"]["net_metering_limit_kw"] = net_metering_limit
    #   5. Load profiles
    site["LoadProfile"] = {"doe_reference_name": "MidriseApartment"}
    # Step 6 - host effective tax rate
    site["Financial"] = {"offtaker_tax_pct": "0.0"}
    # Step 7
    obj["Scenario"]["include_climate_in_objective"] = False
    obj["Scenario"]["include_health_in_objective"] = False
    site["Financial"]["co2_cost_us_dollars_per_tonne"] = "185.0"
    site["net_metering_limit_kw"] = None
    site["PV"] = {
        "module_type": 1,  # Premium
        "dc_ac_ratio": "1.35",  # DC to AC size ratio
        "losses": "0.05",  # System losses
        "array_type": 1,  # Rooftop, fixed
        "federal_itc_pct": "0.3",  # Federal PV Incentives & Tax Treatment
    }
    return obj

post = create_input("Melbourne, FL", 1000.0)

matches before and after a step of filtering:  28 2
matches before and after a step of filtering:  2 1


In [178]:
api_response = get_api_results(post=post, 
                               API_KEY=API_KEY, 
                               api_url=root_url, 
                               results_file=outputs / "custom_input_test.json", 
                               run_id=None)
# new_data2 = template.copy()
# extract_info_into_df(new_data2, 34, api_response)
# display(new_data.iloc[34:35])                     # Compare the new data...
# display(template.iloc[34:35])                     # to the old data

main         INFO     Response OK from https://developer.nrel.gov/api/reopt/stable/job/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L.
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/afb35193-e8ed-4bef-9d8a-1b5ff9697fd7/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/custom_input_test.json


In [141]:
json.dump(api_response, open(outputs / "custom_input_test.json", 'w'))

## Run on every row of spreadsheet <a class="anchor" id="run_all"></a>

In [ ]:
id2urdb = {}

In [297]:
# First, get all the URDB id's
id2urdb[34980] = '5d66d5365457a3ee03c68cdd'  # Franklin, TN
id2urdb[12260] = '62d70d02299fe74c2030b72f'  # Richmond County, GA
id2urdb[49180] = '62d84051299fe74c2030b732'  # Salem, NC
for _, row in data.iterrows():
    if row[0] != 'Virginia':
        continue
    id, city, utility = row[2], row[3] + ', ' + row[4], row[5]
    if id in id2urdb.keys():
        continue
    print('Now getting URDB data for city: {:} - {:}'.format(id, city))
    # urdb = get_urdb_data(city, utility_name_hint=utility)
    urdb = get_urdb_data(city, filters=[valid_date])
    id2urdb[id] = urdb['label']
    json.dump(id2urdb, open(outputs / "UtilityChoices.json", 'w'))
print(id2urdb)
json.dump(id2urdb, open(outputs / "UtilityChoices.json", 'w'))

{16740.0: '62d84051299fe74c2030b732', 24860.0: '62d84051299fe74c2030b732', 34980: '5d66d5365457a3ee03c68cdd', 38860.0: '63052f7f09fe8305307477c4', 12260: '62d70d02299fe74c2030b72f', 26620.0: '637560c7da6b6c4dd6025d0f', 16700.0: '62c614ae8e77102d816deb6a', 26900.0: '5f4e57ce5457a3357aebd02a', 37340.0: '62c44d84924e2352a30de5d6', 49180: '62d84051299fe74c2030b732'}


In [309]:
# First, generate all posts
posts = outputs / "posts"
posts.mkdir(exist_ok=True)

print(template.columns[:10])
data = template.copy()
for _, row in data.iterrows():
    if row[0] != 'Virginia':
        continue
    code = row[2]
    city = row[3] + ', ' + row[4]
    net_metering_capacity = row[10]
    utility = row[5]

    post = create_input(city, net_metering_capacity, labelid=id2urdb[code])
    json.dump(post, open(posts / (str(int(code)) + '.json'), 'w'), indent=2)

MultiIndex([('Unnamed: 0_level_0',                       'Point Person', ...),
            ('Unnamed: 1_level_0',                           'Reviewer', ...),
            (            'Inputs',                               'cbsa', ...),
            (            'Inputs',                          'City/MSA ', ...),
            (            'Inputs',                              'State', ...),
            (            'Inputs',                           'Utility ', ...),
            (            'Inputs',               'System Capacity (kW)', ...),
            (            'Inputs',      'Adjusted System Capacity (kW)', ...),
            (            'Inputs',   'State system capacity limit (kW)', ...),
            (            'Inputs', 'Min btw state and utility NEM (kW)', ...)],
           )


In [310]:
# Now run all the queries!
uuids = {}
# Run all posts
for _, row in data.iterrows():
    if row[0] != 'Virginia':
        continue
    code = row[2]
    post = json.load(open(posts / (str(int(code)) + '.json'), 'r'))

    # uuids[code] = get_run_uuid(post=post, API_KEY=API_KEY, api_url=root_url)
    resp = requests.post(root_url + '/job/?api_key=' + API_KEY, json=post)
    run_id = None
    if not resp.ok:
        raise RuntimeError("Status code {}. {}".format(resp.status_code, resp.content))
    else:
        uuids[code] = resp.json()['run_uuid']
print(uuids)

{16740.0: '2b771f78-99f6-4a5a-80af-7a3e714ad7f9', 24860.0: '9be62363-d569-4402-8300-ec570bb52430', 34980.0: '063250cb-54ac-4c4a-80a2-332cadf7119d', 38860.0: '9bc54d5c-38cb-40c7-9773-0878a346c607', 12260.0: '9b377285-0ce7-47e5-9b33-6ece5f1d6f5e', 26620.0: 'eeb00c3e-db63-4c69-9c20-2ee9ca5de40b', 16700.0: '3e979276-300e-4781-a739-97ec1ad494e3', 26900.0: 'b5f794ae-0d14-4438-a78e-ead73d6aa250', 37340.0: '9fcf253f-ba54-4c9a-aab4-99b69113541c', 49180.0: '4f522471-ec67-442b-b275-e6a4bfdb9288'}


In [317]:
json_results_folder = outputs / "results"
json_results_folder.mkdir(exist_ok=True)
for i, row in data.iterrows():
    if row[0] != 'Virginia':
        continue
    code = row[2]

    api_response = get_api_results(post=post,
                                   API_KEY=API_KEY,
                                   api_url=root_url,
                                   results_file=json_results_folder / (str(int(code)) + '.json'),
                                   run_id=uuids[code])

    extract_info_into_df(data, i, api_response)

data.to_excel(outputs / "results.xlsx")

main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/2b771f78-99f6-4a5a-80af-7a3e714ad7f9/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/results/16740.json
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/9be62363-d569-4402-8300-ec570bb52430/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/results/24860.json
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/063250cb-54ac-4c4a-80a2-332cadf7119d/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for results with interval of 5s...
main         INFO     Saved results to outputs/results/34980.json
main         INFO     Polling https://developer.nrel.gov/api/reopt/stable/job/9bc54d5c-38cb-40c7-9773-0878a346c607/results/?api_key=VGZNvNsVcT4bOkCnXWXAIZ8hlxRMpyQeAjGBdy6L for r

In [314]:
# display(data[('Unnamed: 0_level_0', 'Point Person', 'Unnamed: 0_level_2')])
display(data[data[('Unnamed: 0_level_0', 'Point Person', 'Unnamed: 0_level_2')] == 'Virginia'])

Unnamed: 0_level_0 Unnamed: 1_level_0             Inputs  \
         Point Person           Reviewer               cbsa   
   Unnamed: 0_level_2 Unnamed: 1_level_2 Unnamed: 2_level_2   
24           Virginia              Grace            16740.0   
25           Virginia              Grace            24860.0   
26           Virginia              Grace            34980.0   
27           Virginia                Cam            38860.0   
28           Virginia                Cam            12260.0   
29           Virginia                Cam            26620.0   
30           Virginia              Julia            34980.0   
31           Virginia              Julia            38860.0   
32           Virginia              Julia            16700.0   
33           Virginia              Naomi            26900.0   
34           Virginia              Naomi            37340.0   
35           Virginia              Naomi            49180.0   

                                          \
            City/MSA               State   
   Unnamed: 3_level_2 Unnamed: 4_level_2   
24          Charlotte                 NC   
25         Greenville                 SC   
26           Franklin                 TN   
27     South Portland                 ME   
28    Richmond County                 GA   
29         Huntsville                 AL   
30       Murfreesboro                 TN   
31           Portland                 ME   
32   North Charleston                 SC   
33       Indianapolis                 IN   
34          Melbourne                 FL   
35              Salem                 NC   

                                                                          \
                                           Utility  System Capacity (kW)   
                                 Unnamed: 5_level_2   Unnamed: 6_level_2   
24                            Duke Energy Carolinas                20 kW   
25                            Duke Energy Carolinas                20 kW   
26                       Nashville Electric Service                14 kW   
27                           Central Maine Power Co              4999 kW   
28                                    Georgia Power                10 kW   
29                             Huntsville Utilities                50 kW   
30                                     Atmos Energy                 8 kW   
31                           Central Maine Power Co              4999 kW   
32                                 Dominion Energy                 20 kW   
33  AES Indiana (fka Indianapolis Power & Light Co)             1,000 kW   
34                  Florida power and light company              2000 kW   
35                            Duke Energy Carolinas                20 kW   

                                                                   \
   Adjusted System Capacity (kW) State system capacity limit (kW)   
              Unnamed: 7_level_2               Unnamed: 8_level_2   
24                          20.0                             20.0   
25                          20.0                             20.0   
26                          14.0                              0.0   
27                        4999.0                            660.0   
28                          10.0                             10.0   
29                          50.0                              0.0   
30                           8.0                              0.0   
31                        4999.0                            660.0   
32                          20.0                             20.0   
33                        1000.0                              0.0   
34                        2000.0                             10.0   
35                          20.0                             20.0   

                                       \
   Min btw state and utility NEM (kW)   
                   Unnamed: 9_level_2   
24                                 20   
25                                 20   
26            